In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

2025-08-18 13:32:11.573563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755504131.607757  353907 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755504131.621778  353907 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755504131.675996  353907 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755504131.676062  353907 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755504131.676065  353907 computation_placer.cc:177] computation placer alr

In [3]:
with open("datass/Files/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    word_to_vec_map = {}
    word_to_index_map = {}
    index_to_word_map = {}
    index = 1
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_to_vec_map[word] = vector
        word_to_index_map[word] = index
        index_to_word_map[index] = word
        index+=1

In [4]:
def sentence_to_indices(sentence,maxLength = 45):
    m = sentence.shape[0]
    finalArr = np.zeros((m,maxLength),dtype = 'int')
    for i in range(m):
        ind = 0
        for j in sentence[i].lower().split():
            if j in word_to_index_map.keys():
                finalArr[i][ind] = word_to_index_map[j]
                ind+=1
    return finalArr

In [5]:
sentArr = pd.array(["hello","how are you","wassup my boy"])
sentence_to_indices(sentArr)

array([[ 13076,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0],
       [   198,     33,     82,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0],
       [243124,    193,   1607,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0

In [6]:
def pretrained_embedding_layer(word_to_vec_map , index_to_word_map):
    vocab_size = len(word_to_vec_map.keys()) + 1
    random_word = word_to_vec_map['random']
    
    emb_matrix = np.zeros((vocab_size , random_word.shape[0]))
    for idx,word in index_to_word_map.items():
        emb_matrix[idx,:] = word_to_vec_map[word]
    
    embedding_layer = tf.keras.layers.Embedding(vocab_size , random_word.shape[0],trainable=False)
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [7]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map , index_to_word_map)

2025-08-18 13:32:34.566157: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-08-18 13:32:34.566207: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-08-18 13:32:34.566216: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: garvthakral-ROG-Zephyrus-G15-GA503RM
2025-08-18 13:32:34.566221: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: garvthakral-ROG-Zephyrus-G15-GA503RM
2025-08-18 13:32:34.566703: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 570.133.7
2025-08-18 13:32:34.566762: I external/local_xla/xla/stream_executor/cuda/cuda_diagn

In [8]:
def EmojifyV2():
    sentence_indices = tf.keras.layers.Input(shape=(45,), dtype='int32')
    embeddings = embedding_layer(sentence_indices)

    # First LSTM returns sequences → passes info for each timestep
    X = tf.keras.layers.LSTM(units=128, return_sequences=True)(embeddings)
    X = tf.keras.layers.Dropout(0.5)(X)

    # Second LSTM just returns the final hidden state
    X = tf.keras.layers.LSTM(units=128)(X)
    X = tf.keras.layers.Dropout(0.5)(X)

    # Dense hidden layer for richer features
    X = tf.keras.layers.Dense(64, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.5)(X)

    # Output for 19 classes
    X = tf.keras.layers.Dense(20, activation='softmax')(X)

    model = tf.keras.Model(inputs=sentence_indices, outputs=X)
    return model


In [9]:
model =  EmojifyV2()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 45)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 45, 50)         │    20,000,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 45, 128)        │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 45, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │         1,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,232,838 (77.18 MB)

 Trainable params: 232,788 (909.33 KB)

 Non-trainable params: 20,000,050 (76.29 MB)

In [10]:
model.compile(loss = "categorical_crossentropy"  ,optimizer='adam', metrics = ["accuracy"])

In [11]:

from datasets import load_dataset
ds = load_dataset("cardiffnlp/tweet_eval", "emoji")
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [12]:
tf_ds_train = ds['train'].to_tf_dataset()
tf_ds_test = ds['test'].to_tf_dataset()
tf_ds_valid = ds['validation'].to_tf_dataset()

In [13]:
for x in tf_ds_train.take(1):
    print(x['text'].numpy())

b'Sunday afternoon walking through Venice in the sun with @user \xef\xb8\x8f \xef\xb8\x8f \xef\xb8\x8f @ Abbot Kinney, Venice'


2025-08-18 13:32:42.642040: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
# maxLength = 0
# for x in tf_ds_train:
#     maxLength = max(maxLength,len(x['text'].numpy().split()))
# print(maxLength)

In [15]:
# maxy = 0
# for x in tf_ds_train:
#     # print(tf.one_hot(x['label'],depth = 10))
#     maxy = max(maxy,x['label'].numpy())
# print(maxy)

In [16]:
lookupTable = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(list(word_to_index_map.keys()),list(word_to_index_map.values())),
    default_value = -1
)
def preprocess_data(example,max_len = 45,pad_value = 0):
    example['text'] = tf.strings.lower(input = example['text'])
    example['text'] = tf.strings.split(input = example['text'])
    example['text'] = tf.ragged.map_flat_values(
        lookupTable.lookup,
        example['text']
    )
    example['text'] = example['text'][:max_len]
    
    # Pad if shorter than max_len
    current_len = tf.shape(example['text'])[0]
    pad_len = max_len - current_len
    example['text'] = tf.pad(example['text'], [[0, pad_len]], constant_values=pad_value)
    example['label']  = tf.one_hot(example['label'],depth = 20)
    return (example['text'] , example['label'])
for x,y in tf_ds_train.take(1).map(preprocess_data):
    print(x)
# tf_ds_train.take(1).map(preprocess_data)

tf.Tensor(
[  241  1503  3527   132  8082     7     1  1663    18    -1    -1    -1
    -1 17528 16326    -1  8082     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0], shape=(45,), dtype=int32)


2025-08-18 13:32:43.624609: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
tf_ds_train = tf_ds_train.map(preprocess_data).batch(34).prefetch(tf.data.AUTOTUNE) 
tf_ds_valid = tf_ds_valid.map(preprocess_data).batch(34).prefetch(tf.data.AUTOTUNE) 
tf_ds_test = tf_ds_test.map(preprocess_data).batch(34).prefetch(tf.data.AUTOTUNE) 

In [18]:
model.fit(tf_ds_train , epochs = 50, batch_size = 34)

Epoch 1/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 95s 68ms/step - accuracy: 0.1815 - loss: 2.7959
Epoch 2/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 89s 67ms/step - accuracy: 0.2062 - loss: 2.7036
Epoch 3/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 83s 62ms/step - accuracy: 0.2322 - loss: 2.6406
Epoch 4/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 81s 61ms/step - accuracy: 0.2450 - loss: 2.6055
Epoch 5/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 84s 64ms/step - accuracy: 0.2561 - loss: 2.5671
Epoch 6/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 85s 64ms/step - accuracy: 0.2664 - loss: 2.5270
Epoch 7/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 89s 67ms/step - accuracy: 0.2765 - loss: 2.4932
Epoch 8/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 62s 46ms/step - accuracy: 0.2824 - loss: 2.4649
Epoch 9/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 54s 41ms/step - accuracy: 0.2897 - loss: 2.4407
Epoch 10/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 53s 40ms/step - accuracy: 0.2969 - loss: 2.4149
Epoch 11/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 52s 39ms/step - accuracy: 0.3033 - loss: 2.3907
Epoch 12

In [19]:
model.save('my_model.keras')

In [20]:
model = tf.keras.models.load_model("my_model.keras")
model.compile(loss = "categorical_crossentropy" , optimizer = "adam" , metrics = ['accuracy'])

2025-08-18 14:21:32.705568: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 80000200 exceeds 10% of free system memory.


In [27]:
model.evaluate(tf_ds_valid)

148/148 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2276 - loss: 3.8696


[3.900615692138672, 0.22259999811649323]

In [22]:
label_to_emoji = {
    0: "❤",
    1: "😍",
    2: "😂",
    3: "💕",
    4: "🔥",
    5: "😊",
    6: "😎",
    7: "✨",
    8: "💙",
    9: "😘",
    10: "📷",
    11: "🇺🇸",
    12: "☀",
    13: "💜",
    14: "😉",
    15: "💯",
    16: "😁",
    17: "🎄",
    18: "📸",
    19: "😜"
}


In [23]:
def predict_emoji(predArr,predictions):
    for i in range(len(predictions)):
        print(f"{predArr[i]} {label_to_emoji[np.argmax(predictions[i])]}")

In [24]:
origArr = ["I hate you","Click a picture with me"]
predArr = sentence_to_indices(np.array(origArr))

In [25]:
predictions = model.predict(predArr)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step


In [26]:
predict_emoji(origArr,predictions)

I hate you 😂
Click a picture with me 😂
